In [3]:
import numpy as np
import yfinance as yf
import requests_cache

session = requests_cache.CachedSession('yfinance.cache')

In [4]:
qqq = yf.Ticker("QQQ", session=session)
qqq_hist = qqq.history(period="10y")

voo = yf.Ticker("VOO", session=session)
voo_hist = voo.history(period="10y")

tqqq = yf.Ticker("TQQQ", session=session)
tqqq_hist = tqqq.history(period="10y")

In [21]:
# Compute $\alpha$ and $\sigma^2$ for daily series

def _daily_continuously_compound_return(df, column='Close'):
    series = df['Close'].to_numpy()
    # $X(t) = r(t) = P(t) / P(t-1)$
    return np.log(series[1:] / series[:-1])

def daily_alpha(df):
    r = _daily_continuously_compound_return(df)
    return np.mean(r)

def daily_variance(df):
    r = _daily_continuously_compound_return(df)
    return np.var(r)

def daily_mu(df):
    alpha = daily_alpha(df)
    variance = daily_variance(df)
    # $\mu = \alpha + 1/2*\sigma^2$
    return alpha + 0.5 * variance

def daily_covariance(df_list):
    # r_mat [len(df_list), observations]
    r_mat = np.concatenate([np.reshape(_daily_continuously_compound_return(df), [1,-1]) for df in df_list], axis=0)
    return np.cov(r_mat, rowvar=True)

def daily_portfolio_variance(df_list, weights):
    cov = daily_covariance(df_list)
    w = np.reshape(weights, [1, -1])
    return np.matmul(np.matmul(w, cov), w.T)[0][0]

def daily_portfolio_mu(df_list, weights):
    mu = np.matmul(np.asarray([daily_mu(df) for df in df_list]), np.asarray(weights).T)
    return mu

In [22]:
daily_alpha(qqq_hist), daily_variance(qqq_hist), daily_mu(qqq_hist)

(0.0006381047450368758, 0.00016488066554416715, 0.0007205450778089594)

In [23]:
daily_portfolio_variance([qqq_hist, voo_hist, tqqq_hist], [0.33, 0.33, 0.33])

0.0004099034472738794

In [24]:
daily_portfolio_mu([qqq_hist, voo_hist, tqqq_hist], [0.33, 0.33, 0.33])

0.001085763047522761